In [2]:
import importlib
from lib import kpi

In [9]:
importlib.reload(kpi)

<module 'lib.kpi' from 'c:\\Users\\okpokam\\OneDrive - Queensland University of Technology\\SAEF KPI Reporting\\lib\\kpi.py'>

In [11]:
data           = kpi.load_data()
rpt_config     = kpi.get_rpt_args()
saef_library   = kpi.get_saef_library()
responses_json = kpi.split_response(data[0])
ppl_collection = kpi.person_construct(responses_json, data[2], rpt_config[0])
proj_saef      = kpi.project_construct(responses_json, ppl_collection[0])
buckets        = kpi.matched_library(saef_library, ppl_collection[1])
templates      = kpi.load_templates()
meta_bucket    = buckets[0]
bucket         = buckets[1]
ppl_saef       = dict(sorted(ppl_collection[0].items(), key = lambda x: x[1].get('LastName')))
ppl_hash       = ppl_collection[1]
bulk_responses = data[0]
biblio         = data[1]
scopus         = data[2]
rpt_year       = rpt_config[0]
organisations  = rpt_config[1]
org_shortnames = list(organisations.keys())
proj_saef_nohold = proj_saef[proj_saef.Status != 'On hold']

c:\Users\okpokam\Miniconda3\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [12]:
# 1.remove empty rows
bucket = bucket[bucket['id_person'].isna() == False]
# 2. Make parsing more straight forward
for prsn in ppl_saef:
    if ppl_saef[prsn]['Gender'] == 'Non-binary/Gender diverse' or  ppl_saef[prsn]['Gender']  == 'Prefer not to say':
        ppl_saef[prsn]['Gender'] = 'Other'
# 3. Exclude: Nicole Webster, 027E2DEA-DB06-3946-B8C8-E053EF8E09F0
del ppl_saef['027E2DEA-DB06-3946-B8C8-E053EF8E09F0']; del ppl_hash['NicoleWebster']
# 4. Remove leading/trialing whitespace from  blibliographic entry.
biblio['Biblio'] = biblio['Biblio'].str.strip()
# 5. Jump into your pedantry
for prsn in ppl_saef:
    if ppl_saef[prsn]['Position'] != 'PhD Student' and  ppl_saef[prsn]['Position'] != 'Masters Student' and \
       ppl_saef[prsn]['Position'] != 'Honours Student':
        ppl_saef[prsn]['StudentProjectTitle'] = 'Not applicable'

In [ ]:
# individual
for org in org_shortnames:
    for prsn in ppl_saef:
        p = kpi.get_context_idv(org, ppl_saef, prsn, bucket, biblio, rpt_year)
        if len(p) > 0:
            templates['doc_idv'].render(kpi.get_context_idv(org, ppl_saef, prsn, bucket, biblio, rpt_year))
            templates['doc_idv'].save(f"output/annual/{rpt_year}/idv/{org}_{ppl_saef.get(prsn)['LastName']}{ppl_saef.get(prsn)['FirstName']}.docx") 

In [ ]:
# Organisation
for org in org_shortnames:
    templates['doc_org'].render(kpi.get_context_org(org, organisations, ppl_saef, bucket, biblio, rpt_year, proj_saef_nohold)[0])
    templates['doc_org'].save(f"output/annual/{rpt_year}/org/{org}.docx")

In [84]:
# Program (SAEF) trail run file
# kpi.get_context_saef(org_shortnames[0:7], ppl_saef, bucket, biblio, meta_bucket, rpt_year, proj_saef)
# kpi.sri_researcher_count(kpi.get_context_org('WA Museum', ppl_saef, bucket, biblio, rpt_year, proj_saef)[1], 'Post Doc')
# Program
templates['saef'].render(kpi.get_context_saef(organisations, ppl_saef, bucket, biblio, meta_bucket, rpt_year, proj_saef_nohold))
templates['saef'].save(f"output/annual/Organisation output test/SAEF.docx")

## Draft 1 Debug

In [26]:
# for x in responses_json['People_Detail']['data']:
#     if x['fieldData']['Position'] == 'Associate Investigator' and x['fieldData']['Organisations 2::ShortName'] == 'Monash' \
#     and  x['fieldData']['State'] == 'Active':
#         print(x['fieldData']['LastName'] )
# kpi.get_context_saef(organisations, ppl_saef, bucket, biblio, meta_bucket, rpt_year, proj_saef)
# kpi.get_context_org('QUT', organisations, ppl_saef, bucket, biblio, rpt_year, proj_saef_nohold)[0]
x = kpi.get_context_idv('WA Museum', ppl_saef, ppl_hash.get('NeridaWilson'), bucket, biblio, rpt_year)
# if x.get('Supervision'):
#     for reportee in x.get('Supervision'):
#         print(f"{ppl_saef.get(reportee)['Position']}: {ppl_saef.get(reportee)['Salutation']}")
# scopus['orcid']
print(x)

# ppl_collection[0]
# proj_saef
# set(meta_bucket.key) - set(biblio.ID_Zotero)



{'Salutation': 'Dr Nerida Wilson', 'Position': 'Partner Investigator', 'Organisation': 'Western Australian Museum', 'StartDateDMY': '14/06/2021', 'EndDateDMY': '30/06/2023', 'Fte': 10, 'Profile': 'Dr Nerida Wilson is a research scientist at the Western Australian Museum, Perth, and manages its Molecular Systematic Unit. Her research interests focus on understanding and describing the extent of biological diversity present on earth and resolving the evolutionary relationships among those taxa. She is passionate about taxonomy and systematics as the foundational discipline that underpins biological science.', 'StudentProjectTitle': 'Not applicable', 'Saef_funded': 'No', 'ProjectCodeFTEList': 'T2_P015 (5), T2_P020 (10)', 'CareerStage': 'Senior Researcher', 'CrossnodeSupervision': 'No', 'Workshop': None, 'Supervision': None, 'lastname': 'Wilson', 'Prize': None, 'Scar': None, 'Advisory': 'Research Advisory Committee member, National Marine Facility (2021). https://mnf.csiro.au/en/About/Gove

In [12]:
# kpi.get_org_biblio(biblio, bucket, 'newspaperArticle')
# kpi.matched_library(bucket, ppl_hash.get('StevenChown'))
# ppl_saef[ppl_hash.get('ChrisGreening')]['LastName']
# proj_saef[proj_saef.Status != 'On hold']
# bucket[(bucket.itemType == 'presentation')]
# bucket['ignore'] = bucket['tags'].apply(len)
# pr = bucket[(bucket.itemType == 'presentation') & (bucket.loc[:, ('tags')].apply(len)==0)]
# pr
ppl_saef.get(ppl_hash.get('KerrieWilson'))

# ignore_zot_tags = pr[pr.ignore == 0].key.to_list()
# ignore_zot_tags
# bucket[bucket.key.isin(ignore_zot_tags)]



{'Title': 'Professor',
 'FirstName': 'Kerrie',
 'LastName': 'Wilson',
 'CareerStage': 'Senior Researcher',
 'State': 'Active',
 'FTE': 10,
 'Position': 'Chief Investigator',
 'Gender': 'Woman',
 'StartDate': '2021-06-14',
 'EndDate': '2028-12-31',
 'Org': 'QUT',
 'Organisation': 'Queensland University of Technology',
 'Profile': 'Professor Kerrie Wilson is the Pro Vice-Chancellor (Sustainability Strategy) at Queensland University of Technology (QUT). Kerrie is a leading interdisciplinary decision scientist with more than 190 research publications focusing on applied resource allocation problems, such as where to invest limited resources to protect or restore biodiversity and what factors influence success. She has developed and applied decision science to inform investments in environmental protection, management, and restoration initiatives.',
 'SAEFFunded': 'No',
 'Consent': 'Yes - email, address and phone',
 'Orcid': '0000-0002-0092-935X',
 'Grants': [],
 'Training': [],
 'Prizes': 

In [15]:
biblio

,ID_Zotero,Biblio
0,AJ5ES32P,"Akinlotan, M. D., Warne, D. J., Helmstedt, K. J., Vollert, S. A., Chadès, I., Heneghan, R. F., Xiao, H., & Adams, M. P. (2024). Beyond expected values: Making environmental decisions using value of information analysis when measurement outcome matters. Ecological Indicators, 160, 111828. DOI: https://doi.org/10.1016/j.ecolind.2024.111828"
1,6STGMD7P,"Alinejadtabrizi, T., Lang, F., Huang, Y., Ackermann, L., Keywood, M., Ayers, G., Krummel, P., Humphries, R., Williams, A. G., Siems, S. T., & Manton, M. (2024). Wet deposition in shallow convection over the Southern Ocean. Npj Climate and Atmospheric Science, 7, 76. DOI: https://doi.org/10.1038/s41612-024-00625-1"
2,C3IU44DS,"Duffy, G. A., Montiel, F., Purich, A., & Fraser, C. I. (2024). Emerging long-term trends and interdecadal cycles in Antarctic polynyas. Proceedings of the National Academy of Sciences, 121, e2321595121. DOI: https://doi.org/10.1073/pnas.2321595121"
3,9W4NUD6U,"Heidemann, H., Cowan, T., Power, S. B., & Henley, B. J. (2024). Statistical relationships between the interdecadal Pacific oscillation and el niño–southern oscillation. Climate Dynamics, 62, 2499–2515. DOI: https://doi.org/10.1007/s00382-023-07035-8"
4,3TQF57VX,"Jansen, M. A. K., Andrady, A. L., Bornman, J. F., Aucamp, P. J., Bais, A. F., Banaszak, A. T., Barnes, P. W., Bernhard, G. H., Bruckman, L. S., Busquets, R., Häder, D.-P., Hanson, M. L., Heikkilä, A. M., Hylander, S., Lucas, R. M., Mackenzie, R., Madronich, S., Neale, P. J., Neale, R. E., … Zhu, L. (2024). Plastics in the environment in the context of UV radiation, climate change and the Montreal Protocol: UNEP Environmental Effects Assessment Panel, Update 2023. Photochemical & Photobiological Sciences. DOI: https://doi.org/10.1007/s43630-024-00552-3"
5,7N9KNCD8,"Latombe, G., Boittiaux, P., Hui, C., & McGeoch, M. A. (2024). A kernel integral method to remove biases in estimating trait turnover. Methods in Ecology and Evolution, 2041-210X.14246. DOI: https://doi.org/10.1111/2041-210X.14246"
6,S8VP3FJX,"Lee, J. R., Shaw, J. D., Ropert-Coudert, Y., Terauds, A., & Chown, S. L. (2024). Conservation features of the terrestrial Antarctic Peninsula. Ambio. DOI: https://doi.org/10.1007/s13280-024-02009-4"
7,74GEEA3L,"Potapov, A. M., Chen, T.-W., Striuchkova, A. V., Alatalo, J. M., Alexandre, D., Arbea, J., Ashton, T., Ashwood, F., Babenko, A. B., Bandyopadhyaya, I., Baretta, C. R. D. M., Baretta, D., Barnes, A. D., Bellini, B. C., Bendjaballah, M., Berg, M. P., Bernava, V., Bokhorst, S., Bokova, A. I., … Scheu, S. (2024). Global fine-resolution data on springtail abundance and community structure. Scientific Data, 11, 22. DOI: https://doi.org/10.1038/s41597-023-02784-x"
8,WDD67AHL,"Raniga, D., Amarasingam, N., Sandino, J., Doshi, A., Barthelemy, J., Randall, K., Robinson, S. A., Gonzalez, F., & Bollard, B. (2024). Monitoring of Antarctica’s Fragile Vegetation Using Drone-Based Remote Sensing, Multispectral Imagery and AI. Sensors, 24, 1063. DOI: https://doi.org/10.3390/s24041063"
9,B2KG4UWT,"Saunderson, D., Mackintosh, A. N., McCormack, F. S., Jones, R. S., & Van Dalum, C. T. (2024). How Does the Southern Annular Mode Control Surface Melt in East Antarctica? Geophysical Research Letters, 51, e2023GL105475. DOI: https://doi.org/10.1029/2023GL105475"
